In [1]:
import os, sys
import logging
sys.path.append('..')

logger = logging.getLogger('my_project')
logger.setLevel(logging.DEBUG)  # 设置日志级别

# 创建控制台处理器并设置日志级别
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# 创建格式化器并添加到处理器
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)

# 将处理器添加到自定义日志记录器
logger.addHandler(ch)

In [7]:
import re
import fitz  # PyMuPDF
import pandas as pd
import pdfplumber
from typing import Dict

def find_pages_with_keyword(pdf_path, keyword, max_pages=20):
    # 打开PDF文件
    document = fitz.open(pdf_path)
    pages_with_keyword = []

    # 遍历每一页
    for page_num in range(min(len(document), max_pages)):
        page = document.load_page(page_num)
        text = page.get_text()
        if keyword in text:
            pages_with_keyword.append(page_num + 1)  # 页码从1开始

    return pages_with_keyword

def extract_text_from_page(pdf_path, page_number, header_height=0.1, footer_height=0.1):
    """
    从指定页面提取文本内容，并尝试过滤页头和页尾。

    参数:
        pdf_path (str): PDF 文件路径。
        page_number (int): 页码（从1开始）。
        header_height (float): 要过滤的页头高度比例（页面高度的百分比）。
        footer_height (float): 要过滤的页尾高度比例（页面高度的百分比）。

    返回:
        str: 过滤后的页面文本内容。
    """
    with pdfplumber.open(pdf_path) as pdf:
        if page_number < 1 or page_number > len(pdf.pages):
            return f"无效的页码：{page_number}"
        
        page = pdf.pages[page_number - 1]
        width, height = page.width, page.height

        # 定义要裁剪的区域
        cropping_box = (
            0,  # 左边界
            height * header_height,  # 上边界
            width,  # 右边界
            height * (1 - footer_height)  # 下边界
        )

        cropped_page = page.within_bbox(cropping_box)
        text = cropped_page.extract_text()
        
        return text

def filter_strings_with_dots(strings):
    # 定义正则表达式模式，匹配连续出现的6个点
    pattern = re.compile(r'\.{6}')
    filtered_strings = [s for s in strings if pattern.search(s)]
    return filtered_strings

def extract_chapters(text):
    """
    从文本中提取章节名称和起始页码。

    参数:
        text (str): 包含章节信息的文本。

    返回:
        list of dict: 提取的章节信息，每个章节信息包含"chapter"和"start"。
    """
    if type(text) == list:
        text = "\n".join(text)
        
    chapters = []
    
    # 定义两种模式
    pattern_dots = re.compile(r'^(\S.*?)\s*\.+\s*(\d+)$', re.MULTILINE) # 第四节 公司治理....... 72
    pattern_start = re.compile(r'(\d+)\s+(\S.*)', re.MULTILINE)

    # 6个点的正则
    pattern_dot_validate = re.compile(r'\.{6}')
    
    # 如果匹配到连续6个点的模式
    if pattern_dot_validate.search(text):
        print("匹配到文本中有6个点，采用pattern_dots模式")
        matches_dots = pattern_dots.findall(text)
        for chapter, start_page in matches_dots:
            start_page = int(start_page)
            if chapter.strip():
                chapters.append({"chapter": chapter.strip(), "start": start_page})
    # 如果匹配到页码在开头的模式
    else:
        matches_start = pattern_start.findall(text)
        for start_page, chapter in matches_start:
            start_page = int(start_page)
            if chapter.strip():
                chapters.append({"chapter": chapter.strip(), "start": start_page})
    
    return chapters

def get_bookmarks(filepath: str) -> Dict[int, str]:
    """
    读取 PDF 文件中的目录信息。

    参数:
        filepath (str): PDF 文件路径。

    返回:
        list of dict: 提取的章节信息，每个章节信息包含"chapter", "start", "end"。
    """
    bookmarks = []
    with fitz.open(filepath) as doc:
        toc = doc.get_toc()  # [[lvl, title, page, …], …]
        for level, title, page in toc:
            bookmarks.append({'chapter': title, 'start': page, 'level': level})
    return bookmarks

def filter_pages_with_title_keyword(pdf_path, pages):
    filtered_pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in pages:
            page = pdf.pages[page_num - 1]
            text_objects = page.extract_words()
            for obj in text_objects:
                if "目录" in obj['text'] and obj['height'] >= 12:
                    filtered_pages.append(page_num)
                    break
    return filtered_pages
    
def extract_chapter_info_from_pdf(pdf_path, debug=True):
    """
    提取 PDF 中的章节信息。
    1. 先查找包含“目录”的页面
    2. 过滤掉没有连续出现6个点的页面
    3. 提取章节信息

    参数:
        pdf_path (str): PDF 文件路径。
        keyword (str): 用于查找目录的关键字。

    返回:
        list of dict: 提取的章节信息，每个章节信息包含"chapter", "start", "end"。
                      如果未找到包含关键字的页面或未找到包含连续6个点的内容，则返回相应的错误信息。
    """
    chapters = get_bookmarks(pdf_path)
    if len(chapters) > 3: 
        chapters = [item for item in chapters if item['level'] == 1] # 仅保留level==1的item
        return chapters

    keyword = '目录'
    pages = find_pages_with_keyword(pdf_path, keyword)
    if not pages: 
        logger.error(f"未找到包含关键词'{keyword}'的页面。")
        return []
    logger.info(f"find_pages_with_keyword: {pdf_path}, pages: {pages}")
    
    filter_pages_with_title_keyword(pdf_path, pages)
    # pages_content = [extract_text_from_page(pdf_path, page) for page in pages]
    # for page in pages_content:
    #     logger.debug(page)
    #     logger.debug("=====================")

    # chapters = extract_chapters(pages_content)
    # return chapters

# logging.getLogger().setLevel(logging.DEBUG)
# logger.setLevel(logging.INFO)
path = r'.\data\601899_紫金矿业\601899_紫金矿业_紫金矿业集团股份有限公司2023年年度报告_1219390064.pdf'
extract_chapter_info_from_pdf(path, True)

2024-06-10 15:12:27,330 - INFO - find_pages_with_keyword: .\data\601899_紫金矿业\601899_紫金矿业_紫金矿业集团股份有限公司2023年年度报告_1219390064.pdf, pages: [3, 4]


{'text': '战略报告', 'x0': 99.2272, 'x1': 128.2772, 'top': 22.441599999999994, 'doctop': 1706.2215999999999, 'bottom': 29.441599999999994, 'upright': True, 'height': 7.0, 'width': 29.049999999999997, 'direction': 'ltr'}
{'text': '可持续发展报告', 'x0': 209.94619999999998, 'x1': 259.9962, 'top': 22.553600000000074, 'doctop': 1706.3336, 'bottom': 29.553600000000074, 'upright': True, 'height': 7.0, 'width': 50.05000000000001, 'direction': 'ltr'}
{'text': '治理报告', 'x0': 341.9312, 'x1': 370.45619999999997, 'top': 21.930600000000027, 'doctop': 1705.7105999999999, 'bottom': 28.930600000000027, 'upright': True, 'height': 7.0, 'width': 28.524999999999977, 'direction': 'ltr'}
{'text': '财务报告', 'x0': 461.9112, 'x1': 490.4362, 'top': 22.77060000000006, 'doctop': 1706.5506, 'bottom': 29.77060000000006, 'upright': True, 'height': 7.0, 'width': 28.524999999999977, 'direction': 'ltr'}
{'text': '重要提示', 'x0': 55.9842, 'x1': 155.5842, 'top': 89.08059999999989, 'doctop': 1772.8606, 'bottom': 113.0806, 'upright': True,

In [135]:
get_bookmarks(path)

[{'chapter': '最最终版-压缩', 'start': 1, 'level': 1},
 {'chapter': '安永华明(2024)审字第70007899_H01号_MOFED_审计报告(1)',
  'start': 109,
  'level': 1},
 {'chapter': '封底', 'start': 377, 'level': 1}]

In [131]:
import os

def list_pdf_files(folder_path):
    pdf_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# 示例用法
folder_path = '.\data'  # 将此路径替换为你的文件夹路径
pdf_files = list_pdf_files(folder_path)

for path in pdf_files:
    chapters = extract_chapter_info_from_pdf(path, False)
    print(path, chapters)


.\data\000333_美的集团\000333_美的集团_2023年年度报告_1219429374.pdf [{'chapter': '致股东', 'start': 2, 'level': 1}, {'chapter': '第一节 重要提示、目录和释义', 'start': 5, 'level': 1}, {'chapter': '第二节 公司简介和主要财务指标', 'start': 9, 'level': 1}, {'chapter': '一、公司信息', 'start': 9, 'level': 2}, {'chapter': '二、 联系人和联系方式', 'start': 9, 'level': 2}, {'chapter': '三、 信息披露及备置地点', 'start': 9, 'level': 2}, {'chapter': '四、 注册变更情况', 'start': 10, 'level': 2}, {'chapter': '五、 其他有关资料', 'start': 10, 'level': 2}, {'chapter': '六、 主要会计数据和财务指标', 'start': 10, 'level': 2}, {'chapter': '七、 境内外会计准则下会计数据差异', 'start': 11, 'level': 2}, {'chapter': '1、同时按照国际会计准则与按照中国会计准则披露的财务报告中净利润和净资产差异情况', 'start': 11, 'level': 3}, {'chapter': '2、同时按照境外会计准则与按照中国会计准则披露的财务报告中净利润和净资产差异情况', 'start': 11, 'level': 3}, {'chapter': '八、 分季度主要财务指标', 'start': 11, 'level': 2}, {'chapter': '九、 非经常性损益项目及金额', 'start': 11, 'level': 2}, {'chapter': '第三节 管理层讨论与分析', 'start': 13, 'level': 1}, {'chapter': '一、报告期内公司所处行业情况', 'start': 13, 'level': 2}, {'chapter': '二、报告期内公司从事的主要业务', 'sta

In [114]:
from typing import Dict
import fitz  # pip install pymupdf

def get_bookmarks(filepath: str) -> Dict[int, str]:
    # WARNING! One page can have multiple bookmarks!
    bookmarks = []
    with fitz.open(filepath) as doc:
        toc = doc.get_toc()  # [[lvl, title, page, …], …]
        for level, title, page in toc:
            if level == 1:
                bookmarks.append({'chapter': title, 'start': page, 'level': level})
    return bookmarks

path = r'.\data\600519_贵州茅台\600519_贵州茅台_贵州茅台2023年年度报告_1219506510.pdf'
get_bookmarks(path)

[{'chapter': '一、 本公司董事会、监事会及董事、监事、高级管理人员保证年度报告内容的真实性、准确性、完整性，不存在虚假记载、误导性陈述或重大遗漏，并承担个别和连带的法律责任。',
  'start': 2,
  'level': 1},
 {'chapter': '二、 公司全体董事出席董事会会议。', 'start': 2, 'level': 1},
 {'chapter': '三、 天职国际会计师事务所（特殊普通合伙）为本公司出具了标准无保留意见的审计报告。',
  'start': 2,
  'level': 1},
 {'chapter': '四、 公司负责人丁雄军、主管会计工作负责人蒋焰及会计机构负责人（会计主管人员）蔡聪应声明：保证年度报告中财务报告的真实、准确、完整。',
  'start': 2,
  'level': 1},
 {'chapter': '五、 董事会决议通过的本报告期利润分配预案或公积金转增股本预案', 'start': 2, 'level': 1},
 {'chapter': '六、 前瞻性陈述的风险声明', 'start': 2, 'level': 1},
 {'chapter': '七、 是否存在被控股股东及其他关联方非经营性占用资金情况', 'start': 2, 'level': 1},
 {'chapter': '八、 是否存在违反规定决策程序对外提供担保的情况', 'start': 2, 'level': 1},
 {'chapter': '九、 是否存在半数以上董事无法保证公司所披露年度报告的真实性、准确性和完整性', 'start': 2, 'level': 1},
 {'chapter': '十、 重大风险提示', 'start': 2, 'level': 1},
 {'chapter': '十一、 其他', 'start': 2, 'level': 1},
 {'chapter': '第一节 释义', 'start': 4, 'level': 1},
 {'chapter': '第二节 公司简介和主要财务指标', 'start': 4, 'level': 1},
 {'chapter': '第三节 管理层讨论与分析', 'start': 7, 'level': 1},
 {'chapter': '

In [154]:
import fitz  # PyMuPDF
import cv2
import numpy as np
from paddleocr import PaddleOCR

def extract_image_from_pdf(pdf_path, page_number):
    document = fitz.open(pdf_path)
    page = document.load_page(page_number - 1)
    pix = page.get_pixmap()
    image_data = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
    if pix.n == 4:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_BGRA2BGR)
    pix.save('temp.png')
    return image_data

def analyze_layout(image):
    ocr = PaddleOCR(use_angle_cls=True, lang='ch')
    result = ocr.ocr(image, cls=True)
    boxes = [line[0] for line in result[0]]
    column_counts = {'left': 0, 'right': 0}
    for box in boxes:
        center_x = (box[0][0] + box[2][0]) / 2
        if center_x < image.shape[1] / 2:
            column_counts['left'] += 1
        else:
            column_counts['right'] += 1
    if column_counts['left'] > 0 and column_counts['right'] > 0:
        left_to_right_ratio = column_counts['left'] / column_counts['right']
        if 0.5 <= left_to_right_ratio <= 2:
            return "Two columns"
    return "One column"

page_number = 5
image = extract_image_from_pdf(path, page_number)
print('crop image has finished')
layout_type = analyze_layout(image)
print(f"The layout is: {layout_type}")

crop image has finished
[2024/06/10 14:26:46] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jagger/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jagger/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_